# Multivariate Calculus

This notebook contains code for techniques is multivariable calculus using automatic differentiation frameworks and scipy for crucial calculus topics that form the basis of all other calculus topics.



# Day 1: Partial Derivatives through Engineering Necessity

## Learning Objectives

- Compute partial derivatives $∂f/∂x$, $∂f/∂y$ for multivariable functions.
- Interpret partial derivatives as physical rates of change.
- Implement derivatives in PyTorch, JAX, SciPy with 3D visualizations.
- Connect to thermal management in metamaterial design.
---

## Real-World Problem: Chip Cooling

### The Challenge

An electronic chip has temperature distribution `$T(x, y) = 100 * exp(-0.1 * x^2 - 0.2 * y^2) + 50 * sin(0.5 * x) * cos(0.3 * y)$`, where x, y are coordinates (cm). Engineers need to:

1. Identify hotspots where temperature changes rapidly.
2. Calculate cooling rates in x- and y-directions.
3. Ensure temperature $<150°C$.

In [13]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define a temperature-like scalar field T(x, y)
# Combines an exponential decay term and an oscillatory sine-cosine term
def T(x, y):
    return 100 * torch.exp(-0.1 * x**2 - 0.2 * y**2) + 50 * torch.sin(0.5 * x) * torch.cos(0.3 * y)

def partials():
    # Create evenly spaced points for x and y across [-10, 10]
    # requires_grad=True lets PyTorch track operations for differentiation
    x = torch.linspace(-10, 10, 100, requires_grad=True)
    y = torch.linspace(-10, 10, 100, requires_grad=True)

    # Evaluate the scalar field
    Z = T(x, y)

    # Compute ∂T/∂x and ∂T/∂y using autograd
    # Since T is a scalar field in two variables, we backpropagate from Z
    Z.backward(torch.ones_like(Z))

    # Gradient of T w.r.t x
    dT_dx = x.grad

    # Gradient of T w.r.t y
    # (Note: gradients accumulate by default — they should be zeroed out if re-used)
    dT_dy = y.grad

    # Convert from tensors to NumPy arrays for visualization or further analysis
    return dT_dx.detach().numpy(), dT_dy.detach().numpy(), x.detach().numpy(), y.detach().numpy()

# Compute both partial derivatives and the coordinate grids
dT_dx, dT_dy, x, y = partials()

# Display or inspect computed partial derivatives
dT_dx, dT_dy


(array([-7.0205865 , -4.543504  , -2.0637805 ,  0.3654875 ,  2.6926434 ,
         4.868886  ,  6.849438  ,  8.594774  , 10.071558  , 11.253549  ,
        12.12223   , 12.667285  , 12.886848  , 12.787534  , 12.3842325 ,
        11.699725  , 10.76406   ,  9.613774  ,  8.290935  ,  6.842077  ,
         5.317027  ,  3.767726  ,  2.247037  ,  0.8076581 , -0.4988203 ,
        -1.6226593 , -2.516456  , -3.1353793 , -3.437185  , -3.3821945 ,
        -2.933407  , -2.0571527 , -0.72462654,  1.0852727 ,  3.381544  ,
         6.1551213 ,  9.371784  , 12.964878  , 16.829277  , 20.818485  ,
        24.746645  , 28.39701   , 31.537346  , 33.94153   , 35.41503   ,
        35.820637  , 35.1       , 33.286674  , 30.507465  , 26.97069   ,
        22.942625  , 18.715607  , 14.572897  , 10.756366  ,  7.442032  ,
         4.7270947 ,  2.6295767 ,  1.0994139 ,  0.03773117, -0.67991257,
        -1.1806889 , -1.5756121 , -1.9468856 , -2.3421187 , -2.7748504 ,
        -3.2298536 , -3.671257  , -4.0516405 , -4.3

In [14]:
import jax
import jax.numpy as jnp

def partials_jax():

  # define the scalar field
  # combines exponential decay and oscillation just like in pytorch version
  def T(x,y):
    return 100 * jnp.exp(-0.1 * x**2 - 0.2 * y**2) + 50 * jnp.sin(0.5 * x) * jnp.cos(0.3 * y)

  # generate range of x and y values
  x = jnp.linspace(-10, 10, 100)
  y = jnp.linspace(-10, 10, 100)

  # gradient functions for each variable
  # argnums = 0 → ∂T/∂x, argnums = 1 → ∂T/∂y
  dx = jax.grad(T, argnums = 0)
  dy = jax.grad(T, argnums = 1)

  # vmap applies the gradient function elementwise across both arrays
  # this currently pairs x[i] with y[i] → not a full 2D grid, but a 1D diagonal sampling
  dT_dx = jax.vmap(dx, in_axes=(0,0))(x,y)
  dT_dy = jax.vmap(dy, in_axes = (0,0))(x,y)

  # returns 1D arrays of partials evaluated along matching x,y positions
  return dT_dx, dT_dy

partials_jax()


(Array([-7.0205865 , -4.543504  , -2.063769  ,  0.36547622,  2.6926434 ,
         4.868886  ,  6.8494473 ,  8.594774  , 10.071558  , 11.253549  ,
        12.12223   , 12.667282  , 12.886849  , 12.787532  , 12.3842325 ,
        11.699724  , 10.76406   ,  9.613774  ,  8.290932  ,  6.8420734 ,
         5.317027  ,  3.7677226 ,  2.2470315 ,  0.8076581 , -0.4988203 ,
        -1.6226575 , -2.516456  , -3.1353781 , -3.437185  , -3.382194  ,
        -2.933407  , -2.0571513 , -0.72463036,  1.08528   ,  3.3815439 ,
         6.155123  ,  9.371793  , 12.964869  , 16.829283  , 20.818485  ,
        24.746645  , 28.397013  , 31.537352  , 33.94153   , 35.41503   ,
        35.820637  , 35.1       , 33.286674  , 30.507462  , 26.9707    ,
        22.94264   , 18.715614  , 14.5729    , 10.756366  ,  7.4420366 ,
         4.7270937 ,  2.6295786 ,  1.0994167 ,  0.03773403, -0.67991257,
        -1.1806889 , -1.5756121 , -1.9468851 , -2.3421173 , -2.774849  ,
        -3.229853  , -3.671257  , -4.051639  , -4.3

Solving  $4\cdot\sin(3x)cos(2t)$ as a further example.

In [ ]:
def partials_torch():
  Y = lambda x,t: 4 * torch.sin(3*x) * torch.cos(2*t)

  x = torch.arange(1,10,1,requires_grad = True,dtype = torch.float64)
  t = torch.arange(1,10,1,requires_grad = True, dtype = torch.float64)

  Z = Y(x,t)

  Z.backward(torch.ones_like(Z))
  return x.grad,t.grad

partials_torch()

(tensor([  4.9438,  -7.5313, -10.4981,  -1.4734,   7.6492,   6.6865,  -0.8987,
          -4.8746,  -2.3148], dtype=torch.float64),
 tensor([-1.0266, -1.6917,  0.9212,  4.2469,  2.8302, -3.2237, -6.6304, -2.0858,
          5.7458], dtype=torch.float64))

In [ ]:
def partials_jax_question2():

  Y = lambda x,t: 4 * jnp.sin(3*x) * jnp.cos(2*t)
  x = jnp.arange(1,10,1,dtype = jnp.float32)
  t = jnp.arange(1,10,1, dtype = jnp.float32)
  inputs = jnp.stack([x,t],axis = 1)

  dx = jax.grad(Y, argnums = 0)
  dt = jax.grad(Y, argnums = 1)
  dY_dx = jax.vmap(dx, in_axes=(0,0))(x,t)
  dY_dt = jax.vmap(dt, in_axes = (0,0))(x,t)
  return dY_dx,dY_dt

partials_jax_question2()


(Array([  4.943787 ,  -7.5313096, -10.498082 ,  -1.4733694,   7.6491895,
          6.68653  ,  -0.8987397,  -4.874629 ,  -2.3148499], dtype=float32),
 Array([-1.0265604 , -1.6916987 ,  0.92121834,  4.246903  ,  2.8301628 ,
        -3.2236755 , -6.630378  , -2.085752  ,  5.745809  ], dtype=float32))

-

## Real-World Problem: Terrain Navigation

### The Challenge

An autonomous vehicle navigates terrain `H(x, y) = 10 * exp(-0.05 * x^2 - 0.1 * y^2) + 5 * cos(0.2 * x + 0.3 * y)`, where H is height (m). Needs:

1. Steepest path at $(x, y) = (2, 2)$.
2. Rate of change in 45° direction.
3. Optimize path to target elevation.

In [15]:
def gradient_vector_pytorch():

  # Define the scalar field (objective function)
  # Has both exponential decay and cosine oscillations
  H = lambda x, y: 10 * torch.exp(-0.05 * x**2 - 0.1 * y**2) + 5 * torch.cos(0.2 * x + 0.3 * y)

  # Define input variables and tell PyTorch to track their gradients
  x = torch.tensor(2.0, requires_grad=True)
  y = torch.tensor(2.0, requires_grad=True)

  # Evaluate the scalar field
  F = H(x, y)

  # Compute partial derivatives ∂H/∂x and ∂H/∂y using autograd
  F.backward()

  # Store the gradients in a tensor for easier vector operations
  grads = torch.tensor([x.grad, y.grad])

  # Define a unit vector u = (√2/2, √2/2)
  # This represents the direction in which we’ll measure the rate of change
  unit_vector = torch.tensor([(2**0.5)/2, (2**0.5)/2])

  # Directional derivative = ∇H · u
  # Measures how fast H changes in the direction of u
  directional_derivative = torch.dot(grads, unit_vector)

  # ------------------------------
  # Gradient Descent Demonstration
  # ------------------------------

  # Create parameters to be optimized (start values)
  x_opt = torch.tensor(2.0, requires_grad=True)
  y_opt = torch.tensor(2.0, requires_grad=True)

  # Choose optimizer — SGD will iteratively update x_opt and y_opt to minimize H
  optimizer = torch.optim.SGD([x_opt, y_opt], lr=0.1)

  # Optimization loop
  for i in range(1000):
    # Reset gradients to prevent accumulation
    optimizer.zero_grad()

    # Recompute function value for current parameters
    loss = H(x_opt, y_opt)

    # Backpropagate to compute gradients
    loss.backward()

    # Update parameters (move opposite to gradient direction)
    optimizer.step()

  # Return results:
  # gradients of the field, directional derivative, and optimized parameter values
  return (
      grads.detach().numpy(),
      directional_derivative.detach().numpy(),
      x_opt.detach().numpy(),
      y_opt.detach().numpy()
  )

gradient_vector_pytorch()


(array([-1.9390941, -3.4574528], dtype=float32),
 array(-3.8159347, dtype=float32),
 array(4.5022163, dtype=float32),
 array(7.506556, dtype=float32))

In [16]:
def gradient_vector_jax():
  # Define the scalar field (objective function)
  # Combines exponential decay and cosine oscillation
  H = lambda x, y: 10 * jnp.exp(-0.05 * x**2 - 0.1 * y**2) + 5 * jnp.cos(0.2 * x + 0.3 * y)

  # Define scalar inputs (like parameters)
  x = jnp.array(2, dtype=jnp.float32)
  y = jnp.array(2, dtype=jnp.float32)

  # Compute partial derivatives ∂H/∂x and ∂H/∂y at (x, y)
  # jax.grad returns a callable — we evaluate it immediately at (x, y)
  dx = jax.grad(H, argnums=0)(x, y)
  dy = jax.grad(H, argnums=1)(x, y)

  # Combine partials into gradient vector
  grads = jnp.array([dx, dy])

  # Define the direction vector (unit vector)
  u = jnp.array([(2**0.5)/2, (2**0.5)/2], dtype=jnp.float32)

  # Directional derivative = ∇H · u
  directional_derivative = jnp.dot(grads, u)

  # -------------------------------
  # Simple Gradient Descent Routine
  # -------------------------------

  # Initialize parameter vector and learning rate
  params = jnp.array([x, y])
  lr = 0.01

  # Iteratively update parameters using gradient descent
  for i in range(1000):
    # Compute current gradients
    dx = jax.grad(H, argnums=0)(params[0], params[1])
    dy = jax.grad(H, argnums=1)(params[0], params[1])
    grads = jnp.array([dx, dy])

    # Update rule: w = w - lr * ∇H
    params = params - lr * grads

  # Return final gradient, directional derivative, and optimized parameters
  return grads, directional_derivative, params[0], params[1]

gradient_vector_jax()


(Array([ 0.00316883, -0.00661377], dtype=float32),
 Array(-3.815935, dtype=float32),
 Array(4.9638987, dtype=float32),
 Array(7.2003627, dtype=float32))

## Real-World Problem: Chemical Reactor

### The Challenge

A reactor’s concentration $C(T, P) = 0.01 * T^2 + 0.1 * P$ depends on temperature $T(x, y, t) = 300 * e^{(-0.05 * x^2 - 0.03 * y^2 + 0.1 * t)}$ and pressure $P(x, y, t) = 50 + 10 * sin(0.2 * x + 0.3 * y + t)$. Needs:

1. Rate of change $dC/dt \ \text{at} \ (x, y, t) = (1, 1, 0)$.
2. Optimize reactor conditions.
3. Ensure stable reaction rates.

**Questions**:

- How fast does concentration change at $t = 0$?


In [17]:
def multivariate_chain_rule_pytorch():
  # -----------------------------
  # Define scalar functions
  # -----------------------------
  # C depends on T and P (like an output variable)
  C = lambda T, P: 0.01 * T**2 + 0.1 * P

  # T and P themselves depend on x, y, t
  T = lambda x, y, t: 300 * torch.exp(-0.05 * x**2 - 0.03 * y**2 + 0.1 * t)
  P = lambda x, y, t: 50 + 10 * torch.sin(0.2 * x + 0.3 * y + t)

  # -----------------------------
  # Define parameters
  # -----------------------------
  # Set requires_grad=True so PyTorch tracks computation for gradients
  x = torch.tensor(1.0, requires_grad=True)
  y = torch.tensor(1.0, requires_grad=True)
  t = torch.tensor(0.0, requires_grad=True)

  # -----------------------------
  # Compute function values
  # -----------------------------
  # First compute intermediate functions T(x,y,t) and P(x,y,t)
  T_func = T(x, y, t)
  P_func = P(x, y, t)

  # Then compute C(T,P) using these intermediate values
  C_func = C(T_func, P_func)

  # -----------------------------
  # Backpropagate gradients
  # -----------------------------
  # This computes dC/dx, dC/dy, dC/dt using the chain rule automatically
  C_func.backward()

  # -----------------------------
  # Return the derivative w.r.t t
  # -----------------------------
  return t.grad.item()

multivariate_chain_rule_pytorch()


154.2634735107422

In [18]:
def multivariate_chain_rule_jax():
    # -----------------------------
    # Define scalar functions
    # -----------------------------
    C = lambda T, P: 0.01 * T**2 + 0.1 * P
    T = lambda x, y, t: 300 * jnp.exp(-0.05 * x**2 - 0.03 * y**2 + 0.1 * t)
    P = lambda x, y, t: 50 + 10 * jnp.sin(0.2 * x + 0.3 * y + t)

    # -----------------------------
    # Compute derivative w.r.t t
    # -----------------------------
    # Wrap the chain of functions into a single callable (lambda)
    # Now jax.grad can correctly compute ∂C/∂t
    dt = jax.grad(
        lambda x, y, t: C(T(x, y, t), P(x, y, t)),  # function of three variables
        argnums=2  # derivative w.r.t t
    )(1.0, 1.0, 0.0)  # evaluate at x=1, y=1, t=0

    return dt

multivariate_chain_rule_jax()


Array(154.26347, dtype=float32, weak_type=True)

# Multiple Integrals
### The Challenge

A metamaterial slab has density `ρ(x, y) = 2 + 0.5 * sin(0.2 * x) * cos(0.3 * y)` g/cm² over $0 ≤ x ≤ 5, 0 ≤ y ≤ 4 (cm)$.
 Needs:

1. Total mass.

**Questions**:

- What is the total mass?

- worked example by hand yielded 43.58g

In [19]:
import numpy as np
from scipy.integrate import dblquad

# -----------------------------
# Define the density function
# -----------------------------
# rho(y,x) gives the mass density at a point (x,y) on the slab
rho = lambda y, x: 2 + 0.5 * np.sin(0.2 * x) * np.cos(0.3 * y)

# -----------------------------
# Compute total mass via double integral
# -----------------------------
# dblquad integrates a function over a rectangular region:
#   first argument: function to integrate (rho)
#   second & third: x-limits (a, b)
#   fourth & fifth: y-limits as functions of x (here constant 0 to 4)
total_mass, error = dblquad(
    rho,          # function to integrate
    0, 5,         # x goes from 0 to 5
    lambda x: 0,  # y lower limit
    lambda x: 4   # y upper limit
)

# -----------------------------
# Print result
# -----------------------------
print(f"The total mass of the slab is {total_mass:.2f} g")


The total mass of the slab is 43.57 g


# Lagrange Multipliers
## Real-World Problem: Antenna Design

### The Challenge

An antenna’s gain is $G(x, y) = 10 - 0.5 * x^2 - 0.3 * y^2 + 0.2 * x * y$, with constraint $x + y = 8 \ \text{cm}$
Needs:

1. Maximize gain G.
2. Verify optimal dimensions.
3. Ensure computational stability.

**Questions**:

- What dimensions maximize gain?
- Does it satisfy x + y = 8?  

Why It Matters: Constrained optimization balances properties in topology optimization research.

In [20]:
from scipy.optimize import minimize

# --------------------------------------
# Define the objective function
# --------------------------------------
# G(x, y) represents a gain function we want to maximize.
# Since scipy.minimize() minimizes by default, we negate the function.
G = lambda xy: -(10 - 0.5 * xy[0]**2 - 0.3 * xy[1]**2 + 0.2 * xy[0] * xy[1])

# --------------------------------------
# Define the constraint
# --------------------------------------
# The constraint is an equality: x + y = 8
# 'type': 'eq' means the constraint must equal zero → xy[0] + xy[1] - 8 = 0
constraint = {'type': 'eq', 'fun': lambda xy: xy[0] + xy[1] - 8}

# --------------------------------------
# Perform optimization
# --------------------------------------
# initial guess = [1,1]
# method = 'SLSQP' (Sequential Least Squares Quadratic Programming)
# used for problems with equality and inequality constraints
result = minimize(G, [1, 1], constraints=constraint, method='SLSQP')

# unpack optimized values
x, y = result.x

# --------------------------------------
# Display the optimal values and result
# --------------------------------------
print(f"SciPy: Optimal (x, y) = ({x:.2f}, {y:.2f}), Gain = {-result.fun:.2f}")


SciPy: Optimal (x, y) = (3.20, 4.80), Gain = 1.04
